# Feature Extraction

### Import Statements

In [ ]:
import pylidc as pl
from pylidc.utils import consensus
import numpy as np
from keras.models import Model
import pandas as pd
import tensorflow as tf
from cnn_3d_architecture import get_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import statistics

c:\Users\Utilizador\anaconda3\envs\model\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Model Implementation

We implemented a 3D Convolutional Neural Network (CNN) model pre-trained on CT scans, focusing on feature extraction and emphasizing its superiority over traditional 2D approaches. Building on the foundational work of Shen et al. (2015), Causey et al. (2018), and Liu et al. (2018), our model leverages the volumetric nature of CT data to capture spatial relationships across multiple slices, leading to more comprehensive feature representation. This 3D architecture enhances the model's ability to identify complex patterns and structures within the scans, resulting in improved diagnostic performance compared to 2D models that analyze each slice independently.

The model weights were extracted from a pre-trained model developed by Morozov et al. (2020), in which a 3D convolutional neural network was trained to predict presence of pneumonia.

In [5]:
model=get_model()
model.load_weights("model_weights.h5")


Our 3D Convolutional Neural Network (CNN) is specifically designed for feature extraction from CT scan data, accepting input volumes of 128x128x64 with a single channel. The architecture consists of four convolutional blocks, with the first two using 64 filters, the third using 128 filters, and the final block employing 256 filters. Each block is followed by max pooling and batch normalization to enhance feature representation. After the convolutional layers, a global average pooling layer is applied to condense features, followed by a dense layer with 512 units and dropout for regularization. This model is optimized for capturing intricate features in 3D data, leveraging its structure for superior feature extraction compared to 2D models

In [78]:
model_input = model.input
intermediate_layer = model.get_layer('max_pooling3d_35')
intermediate_model = Model(model_input,intermediate_layer.output)

The preprocess_ct_scan function processes a 3D numpy array of CT scan slices by resizing each slice to a uniform dimension of 
128
×
128
128×128. It ensures that the final output contains a total of 64 slices, filling any missing slices with zeros or repeating the last available slice when there are fewer than 64 slices. After resizing and padding, the function adds a batch dimension to the data, reshaping the output into an array with the shape 
(
1
,
128
,
128
,
64
,
1
)
(1,128,128,64,1), making it suitable for input into neural network models.

In [ ]:
def preprocess_ct_scan(ct_scan_slices):
    num_slices = ct_scan_slices.shape[0]

    processed_slices = np.zeros((128, 128, 64, 1))

    for i in range(64):
        if i < num_slices:
            
            resized_slice = np.zeros((128, 128))
            resized_slice[:ct_scan_slices[i].shape[0], :ct_scan_slices[i].shape[1]] = ct_scan_slices[i]
            
            processed_slices[:, :, i, 0] = resized_slice
        else:
            processed_slices[:, :, i, 0] = processed_slices[:, :, min(num_slices - 1, 0), 0]
    
    processed_input = np.expand_dims(processed_slices, axis=0) 
    
    return processed_input

In [79]:
scans_with_annotations = pl.query(pl.Scan).filter(pl.Scan.annotations.any()).all()

features_list = []

nodule_id_counter = 1
df=pd.DataFrame([])
ids=[]
for scan in scans_with_annotations:

    patient_id = scan.patient_id
    
    print(f"Processing Patient ID: {patient_id}")

    nods = scan.cluster_annotations()

    for anns in nods:
        if anns:
            cmask, _, _ = pl.utils.consensus(anns, clevel=0.5, pad=[(20, 20), (20, 20), (0, 0)])
            cmask=preprocess_ct_scan(cmask)
            outputs=intermediate_model.predict(cmask)
            features = np.squeeze(outputs) 
            flattened_features = features.flatten()
            ids.append(patient_id)
            df1=pd.DataFrame([flattened_features])
            df=pd.concat([df,df1])
            print(nodule_id_counter)
            nodule_id_counter += 1   
df.to_csv('output_conv3d_maxpooling.csv', index=False) 

Processing Patient ID: LIDC-IDRI-0078
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
4
Processing Patient ID: LIDC-IDRI-0069
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
7
Processing Patient ID: LIDC-IDRI-0079
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
8
Processing Patient ID: LIDC-IDRI-0101
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
9
Processing Patient ID: LIDC-IDRI-0110
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
11
Processing Patient ID: LIDC-IDRI-0115
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
12
Processing Patient ID: LIDC-IDRI-0132
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [7]:
df = pd.read_csv('output_conv3d_maxpooling.csv')

This code processes all medical scans with annotations for nodules. For each scan, it extracts the patient_id and clusters the annotations. It then calculates various nodule features—such as subtlety, internal structure, and malignancy—using helper functions for mode and mean calculations. The features are stored in a dictionary, which is appended to a list. Finally, the collected features are saved as a CSV file named output_anot.csv.

According to Causey et al. (2018), malignancy rating assigned should be the average of the malignancy ratings assigned by the radiologists
who annotated the nodule, rounded to the nearest integer.

In [ ]:
additional_features = pl.annotation_feature_names

scans_with_annotations = pl.query(pl.Scan).filter(pl.Scan.annotations.any()).all()

features_list = []

nodule_id_counter = 1

for scan in scans_with_annotations:

    patient_id = scan.patient_id
    
    print(f"Processing Patient ID: {patient_id}")

    nods = scan.cluster_annotations()

    for anns in nods:
        if anns:
            features={}
            features['Patient_ID'] = patient_id
            features['Nodule_ID'] = f'Nodule_{nodule_id_counter}'
            nodule_id_counter += 1

            def calculate_value(value):
                try:
                    return statistics.mode(value)
                except statistics.StatisticsError:
                    return np.mean(value)


            def calculate_mean(value):
                return np.mean(value)

            subtlety_value = calculate_value([ann.subtlety for ann in anns])
            internalStructure_value = calculate_value([ann.internalStructure for ann in anns])
            calcification_value = calculate_value([ann.calcification for ann in anns])
            sphericity_value = calculate_value([ann.sphericity for ann in anns])
            margin_value = calculate_value([ann.margin for ann in anns])
            lobulation_value = calculate_value([ann.lobulation for ann in anns])
            spiculation_value = calculate_value([ann.spiculation for ann in anns])
            texture_value = calculate_value([ann.texture for ann in anns])
            malignancy_value = calculate_mean([ann.malignancy for ann in anns])
            for feature_name in additional_features:
                features['subtlety'] = subtlety_value
                features['internalStructure'] = internalStructure_value
                features['sphericity'] = sphericity_value
                features['margin'] = margin_value
                features['lobulation'] = lobulation_value
                features['spiculation'] = spiculation_value
                features['texture'] = texture_value
                features['malignancy'] = malignancy_value

            features_list.append(features)
        
features_df = pd.DataFrame(features_list)

features_df.to_csv('output_anot.csv', index=False)
print("CSV file saved successfully.")

Processing Patient ID: LIDC-IDRI-0078
Processing Patient ID: LIDC-IDRI-0069
Processing Patient ID: LIDC-IDRI-0079
Processing Patient ID: LIDC-IDRI-0101
Processing Patient ID: LIDC-IDRI-0110
Processing Patient ID: LIDC-IDRI-0115
Processing Patient ID: LIDC-IDRI-0132
Processing Patient ID: LIDC-IDRI-0136
Processing Patient ID: LIDC-IDRI-0150
Processing Patient ID: LIDC-IDRI-0151
Processing Patient ID: LIDC-IDRI-0154
Processing Patient ID: LIDC-IDRI-0001
Processing Patient ID: LIDC-IDRI-0002
Processing Patient ID: LIDC-IDRI-0003
Processing Patient ID: LIDC-IDRI-0004
Processing Patient ID: LIDC-IDRI-0005
Processing Patient ID: LIDC-IDRI-0006
Processing Patient ID: LIDC-IDRI-0007
Processing Patient ID: LIDC-IDRI-0008
Processing Patient ID: LIDC-IDRI-0009
Processing Patient ID: LIDC-IDRI-0010
Processing Patient ID: LIDC-IDRI-0011
Processing Patient ID: LIDC-IDRI-0012
Processing Patient ID: LIDC-IDRI-0013
Processing Patient ID: LIDC-IDRI-0014
Processing Patient ID: LIDC-IDRI-0015
Processing P

In [8]:
features_df=pd.read_csv("output_anot.csv")

Finally, the two dataframes were concatenated in order to create the final dataframe containing extracted features using CNN and relevant information from annotations

In [9]:
df_final=pd.concat([features_df,df],axis=1)

In [10]:
df_final

,Patient_ID,Nodule_ID,subtlety,internalStructure,sphericity,margin,lobulation,spiculation,texture,malignancy,...,18422,18423,18424,18425,18426,18427,18428,18429,18430,18431
0,LIDC-IDRI-0078,Nodule_1,4,1,4,4,2,2,5,3.75,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
1,LIDC-IDRI-0078,Nodule_2,5,1,4,2,4,1,5,3.75,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
2,LIDC-IDRI-0078,Nodule_3,4,1,5,5,1,1,5,1.00,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
3,LIDC-IDRI-0078,Nodule_4,5,1,4,2,4,3,5,4.25,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
4,LIDC-IDRI-0069,Nodule_5,3,1,5,5,5,5,5,3.25,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2646,LIDC-IDRI-0639,Nodule_2647,4,1,4,4,2,2,5,4.00,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
2647,LIDC-IDRI-0639,Nodule_2648,1,1,4,1,1,1,1,3.50,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
2648,LIDC-IDRI-0638,Nodule_2649,2,1,4,3,1,1,5,3.50,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0
2649,LIDC-IDRI-0638,Nodule_2650,5,1,4,5,1,1,5,2.00,...,0.0,5.204972,5.560269,0.326125,0.639065,0.0,0.0,4.606907,0.157981,0.0


### Data Normalization

As we know from Lapedes & Farber (1987), Neural networks tend to work better with values in a lower range. 

So, we rescaled the values into a range of [0,1] using Min-Max Scaling.

In [12]:
X = df_final.drop(columns=['Patient_ID','Nodule_ID','malignancy'])
scaler = MinMaxScaler()
X= scaler.fit_transform(X)

### Autoencoder

According to D. Kumar, A. Wong and D. A. Clausi, "Lung Nodule Classification Using Deep Features in CT Images," 2015 12th Conference on Computer and Robot Vision, Halifax, NS, Canada, 2015, pp. 133-138, doi: 10.1109/CRV.2015.25. we decided to implement an autoencoder, a type of neural network designed to learn a compressed representation of data by encoding and subsequently reconstructing it. The autoencoder consists of two primary parts: the encoder, which compresses the input into a lower-dimensional representation, and the decoder, which reconstructs the data from this compressed form. This architecture is highly effective for feature extraction, as it retains essential information while reducing noise. By training on non-linear relationships within data, autoencoders can reveal hidden patterns, enhancing classification tasks such as lung nodule analysis by identifying subtle distinctions in medical imaging data. Here, the structure includes five encoding and decoding layers, optimized with the Adam optimizer and mean squared error loss, which helps achieve robust feature extraction while maintaining reconstruction accuracy.

In [13]:
# Definir o número de features
input_dim = X.shape[1]

# Definir a estrutura do autoencoder (5 camadas)
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)

# Decodificação
decoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(decoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Modelo completo (Autoencoder)
autoencoder = Model(inputs=input_layer, outputs=decoded)

# Compilar o modelo
autoencoder.compile(optimizer='adam', loss='mse')

To improve training efficiency and avoid overfitting, early stopping with a patience of five epochs and dropout regularization were incorporated. Dropout, applied here at a rate of 0.5, randomly drops nodes during training, helping the model generalize better by preventing reliance on specific neurons. After training, we created a dedicated encoder model to extract meaningful, compressed features from the original dataset, capturing essential data characteristics in a lower-dimensional form. These encoded features were then added to the original dataset, enhancing it with a robust, distilled representation that supports more effective subsequent analysis.

In [ ]:
# Early stopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

encoded = Dense(256, activation='relu')(input_layer)
encoded = Dropout(0.5)(encoded)  # Adiciona dropout para prevenir overfitting
encoded = Dense(256, activation='relu')(encoded)

# Treinamento do autoencoder
autoencoder.fit(X, X, 
                epochs=300, 
                batch_size=128, 
                shuffle=True)

# Criar o modelo que apenas usa a parte codificada
encoder = Model(inputs=input_layer, outputs=encoded)

# Extrair as features codificadas
encoded_features = encoder.predict(X)


encoded_df = pd.DataFrame(encoded_features, columns=[f'encoded_feature_{i}' for i in range(encoded_features.shape[1])])

# Concatenar as novas features ao dataframe original
df_final_encoded = pd.concat([df_final, encoded_df], axis=1)

Epoch 1/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.2054
Epoch 2/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0025
Epoch 3/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0018
Epoch 4/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0016
Epoch 5/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0016
Epoch 6/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0015
Epoch 7/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0014
Epoch 8/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0015
Epoch 9/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0011
Epoch 10/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 8.7866e-04
Epoch 11/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 8.8493e-04
Epoch 12/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 7.4405e-04
Epoch 13/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 5.9019e-04
Epoch 14/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 5.8708e-04
Epoch 15/300
21/21 ━━━━━━━━━━━━━━━━━━

In [ ]:
df_final_encoded.to_csv('lung_cancer.csv', index=False)